## Homework 8 - Training a GCN model on the Cora dataset

Please implement the following three functions:
- normalize_adj() - Normalize adjacent matrix
- GraphConvolution() - Design a graph convolution layer
- GCN() - Construct a graph convolutional Network

Please train a two-layer GCN with hidden dimension of 32 on the Cora dataset and print the training results for each epoch.

In [9]:
import time
import math
import argparse
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

from utils import load_data, accuracy


def normalize_adj(adj):
    """Normalize adjacency matrix."""
    # pass
    # Calculate the degree matrix
    degree = np.array(adj.sum(1))
    degree_inv_sqrt = np.power(degree, -0.5).flatten()
    degree_inv_sqrt[np.isinf(degree_inv_sqrt)] = 0.  # Handle division by zero
    degree_inv_sqrt_matrix = np.diag(degree_inv_sqrt)

  # Convert NumPy arrays to PyTorch tensors
    adj_tensor = torch.tensor(adj)
    degree_inv_sqrt_tensor = torch.tensor(degree_inv_sqrt_matrix)

    # Normalize adjacency matrix
    normalized_adj_tensor = adj_tensor.mm(degree_inv_sqrt_tensor).t().mm(degree_inv_sqrt_tensor)

    return normalized_adj_tensor

# Seed radom seed
np.random.seed(42)
torch.manual_seed(42)

# Load dat
adj, features, labels, idx_train, idx_val, idx_test = load_data()
adj = normalize_adj(adj)

Loading cora dataset...


/tmp/ipykernel_508787/2454436645.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adj_tensor = torch.tensor(adj)


In [10]:
class GraphConvolution(Module):
    """Simple GCN layer, similar to https://arxiv.org/abs/1609.02907"""
    
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        # Tip
        # pass
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)


    def forward(self, x, adj):
        # Tip
        # pass
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        if self.bias is not None:
            output = output + self.bias
        return output

        
class GCN(nn.Module):
    '''GCN model with two GCN layers'''
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        # Tip
        # pass
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, adj):
        # Tip
        # pass
        x = F.relu(self.gc1(x, adj))
        x = self.dropout(x)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)



def train(epoch):
    '''Train the model on the training data'''
    
    t = time.time()
    model.train()
    optimizer.zero_grad()
    
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # Evaluate the model on the val data
    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    
    print('Epoch: {:04d}'.format(epoch + 1),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()),
        'loss_val: {:.4f}'.format(loss_val.item()),
        'acc_val: {:.4f}'.format(acc_val.item()),
        'time: {:.4f}s'.format(time.time() - t))

    
def test():
    '''Evaluate the model on the testing data'''
    
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:", "loss= {:.4f}".format(loss_test.item()), "accuracy= {:.4f}".format(acc_test.item()))

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.05,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

args = parser.parse_args(args=[])
args.cuda = torch.cuda.is_available()
if args.cuda:
    torch.cuda.manual_seed(42)

# Model and optimizer
model = GCN(nfeat=features.shape[1], nhid=args.hidden, nclass=labels.max().item() + 1, dropout=args.dropout)
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# Load data into CUDA
if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")

# Testing
test()

Epoch: 0001 loss_train: 1.8859 acc_train: 0.2857 loss_val: 1.8185 acc_val: 0.3500 time: 0.5230s
Epoch: 0002 loss_train: 1.8213 acc_train: 0.2929 loss_val: 1.7876 acc_val: 0.3500 time: 0.0025s
Epoch: 0003 loss_train: 1.7685 acc_train: 0.3000 loss_val: 1.7714 acc_val: 0.3500 time: 0.0014s
Epoch: 0004 loss_train: 1.7298 acc_train: 0.3429 loss_val: 1.7463 acc_val: 0.4167 time: 0.0013s
Epoch: 0005 loss_train: 1.6763 acc_train: 0.4214 loss_val: 1.7088 acc_val: 0.4367 time: 0.0013s
Epoch: 0006 loss_train: 1.6324 acc_train: 0.4429 loss_val: 1.6676 acc_val: 0.4533 time: 0.0013s
Epoch: 0007 loss_train: 1.5858 acc_train: 0.4714 loss_val: 1.6227 acc_val: 0.4833 time: 0.0013s
Epoch: 0008 loss_train: 1.5134 acc_train: 0.5286 loss_val: 1.5744 acc_val: 0.5000 time: 0.0013s
Epoch: 0009 loss_train: 1.4487 acc_train: 0.5500 loss_val: 1.5252 acc_val: 0.5067 time: 0.0015s
Epoch: 0010 loss_train: 1.3891 acc_train: 0.5429 loss_val: 1.4753 acc_val: 0.4967 time: 0.0015s
Epoch: 0011 loss_train: 1.2775 acc_train